In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [2]:
%run for_rev_scaling_ALGs.ipynb

### TU

#### Generate a parametrized example

In [13]:
np.random.seed(10870)
n_i = 300
m_j = 310
k = 2

θ_true = np.array([3])

# X_i = np.random.normal([5,5],[5,2], size = [n_i, k]).round(16)
# Y_j = np.random.normal([5,5],[5,2], size = [m_j, k]).round(16)

X_i = np.random.normal(3,3, size = n_i).round(0)
Y_j = np.random.normal(3,3, size = m_j).round(0)
#X_ijk = np.random.choice(10, size = (n_i,m_j, k))

#A = np.exp( np.abs((X_i[:,None,:]- Y_j[None,:,:]))@  θ_true[:k])
A = np.ones([n_i, m_j])
B = X_i[:,None]*Y_j[None,:] * θ_true

obs_mrk = OneToOneITU( n_i,m_j, (A,B))
eq = mu_obs, u_true, v_obs = obs_mrk.forward_backward_scaling(5,1e-10,1/4)

In [14]:
obs_mrk.check_all(eq)

FEAS
i: True, j:  True
#matched: 261 over 300
________________
ε-CS
min_ij D_ij : -3.637978807091713e-11 
________________
IR
i: 0.0, j:  0.0


In [24]:
def simulation(self,θ, n_simul):
    A_θ = np.ones([self.n,self. m])
    B_θ = X_i[:,None]*Y_j[None,:] * θ

    np.random.seed(123)
    epsilons = np.random.normal(0,1,size = [n_simul, self.n, self.m])
    results = np.zeros([self.n, self.m])
    for r in range(n_simul):
        mkt_r = OneToOneITU( self.n, self.m, (A_θ,B_θ + epsilons[r]))
        mu_r, u_r, v_r = mkt_r.forward_backward_scaling(5,1e-8,1/4)
        results += mu_r
    
    return results/ n_simul

OneToOneITU.simulation = simulation

In [25]:
obs_mrk.simulation(2, 10)

array([[0. , 0. , 0. , ..., 0. , 0. , 0.1],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

## estimator

In [7]:
def Q(self,θ):
    A_θ = np.ones([self.n,self. m])
    B_θ = X_i[:,None]*Y_j[None,:] * θ

    mkt_θ = OneToOneITU( self.n,self.m, (A_θ,B_θ))   
    mu_θ, u_θ, v_θ = mkt_θ.forward_backward_scaling(5,1e-10,1/4)

    Q_θ = 
 
    
    return Q_θ

OneToOneITU.Q = Q

In [8]:
θ_true

array([ 0.2, -0.2,  1. ,  2. ])

In [9]:
obs_mrk.Q( θ_true )

TypeError: OneToOneITU.__init__() got an unexpected keyword argument 'tol'

In [ ]:
def maximum_score_estimator(self,θ_0 ):

    problem = sp.optimize.minimize( self.Q ,θ_0,method= 'BFGS' ,options= {'gtol' : 1e-7 })  

    return problem.x, problem.fun

OneToOneITU.maximum_score_estimator = maximum_score_estimator

In [ ]:
results = obs_mrk.maximum_score_estimator( np.array([0.5,-0.5,0,0])  )

[ 0.5 -0.5  0.   0. ]
CS0.0
974.1989266368079
978.8040968227959
[ 0.5 -0.5  0.   0. ]
CS0.0
974.1989266368079
978.8040968227959
[ 0.5 -0.5  0.   0. ]
CS0.0
974.1989266368079
978.8040968227959
[ 0.5 -0.5  0.   0. ]
CS-0.0022961327955788955
974.195594129959
978.7982541403304
[ 0.5 -0.5  0.   0. ]
CS-0.0005170533067114087
974.1954162699465
978.8012757794195
[ 0.5   -0.5    0.91   0.439]
CS-7.2960933245068665e-06
671.4551782128166
692.4496616508436
[ 0.5   -0.5    0.91   0.439]
CS-7.29609370642796e-06
671.4537841372261
692.4477209970069
[ 0.5   -0.5    0.91   0.439]
CS-7.296093180246282e-06
671.4549092503684
692.4493824657176
[ 0.5   -0.5    0.91   0.439]
CS-7.296093324277162e-06
671.4548508751968
692.4493236621031
[ 0.5   -0.5    0.91   0.439]
CS-7.296093324392014e-06
671.4554634819098
692.4499552250724
[ 0.504 -0.499  0.911  0.439]
CS-6.055578736158636e-05
681.3053849097209
702.340194891482
[ 0.504 -0.499  0.911  0.439]
CS-6.055657553700281e-05
681.3005132266869
702.3346062192776
[ 0.504

KeyboardInterrupt: 

In [ ]:
# num_rows = 2**k  # 16 rows
# num_cols = k  # 4 columns

# # Generate the binary representation matrix in a concise way
# directions = np.array(np.unpackbits(np.arange(num_rows, dtype=np.uint8).reshape(-1, 1), axis=1)[:, -num_cols:], dtype= int)
# directions = np.concatenate((directions, -directions), axis= 0)

In [ ]:
# opt = (np.zeros(k), np.inf)

# for d in directions:
#     if np.abs(d[0] + d[1])<2:
#         results_d = obs_mrk.maximum_score_estimator( d, mu_obs,X_ijk )
#         print(f"direction: {d}")
#         print(results_d)
#         if results_d[1] < opt[1]:
#             opt = results_d